In [1]:
from facenet_pytorch import InceptionResnetV1
import torch
import os
from PIL import Image
from torchvision import transforms
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Wczytanie modelu FaceNet
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Funkcja do przetwarzania obrazów
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Zwiększ rozdzielczość
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0).to(device)

def get_embedding(image_path):
    img = preprocess_image(image_path)
    with torch.no_grad():
        embedding = model(img)
    return embedding.detach().cpu().numpy().flatten()


In [2]:
def get_all_embeddings(folder_path):
    embeddings = {}
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        try:
            embeddings[img_name] = get_embedding(img_path)
        except Exception as e:
            print(f"Problem z przetwarzaniem obrazu {img_name}: {e}")
    return embeddings


In [3]:
def calculate_similarities(embeddings):
    image_names = list(embeddings.keys())
    similarities = []

    for i in range(len(image_names)):
        for j in range(i + 1, len(image_names)):
            embedding1 = embeddings[image_names[i]]
            embedding2 = embeddings[image_names[j]]
            similarity = cosine_similarity([embedding1], [embedding2])[0][0]
            similarities.append((image_names[i], image_names[j], similarity))

    return similarities


In [7]:
folder_path = "C:\\Users\\Acer Nitro 5\\Desktop\\STUDIA\\SEMESTR4\\DE\\ML\\projektowe_hachaj\\dataset2\\mfr2\\ZuzanaCaputova"  # Zamień na rzeczywistą ścieżkę do folderu


from sklearn.preprocessing import normalize

# Normalizacja embeddingów
def normalize_embeddings(embeddings):
    return {key: normalize([value])[0] for key, value in embeddings.items()}


# Generowanie embeddingów
embeddings = get_all_embeddings(folder_path)
embeddings = normalize_embeddings(embeddings)

# Obliczanie podobieństw
similarities = calculate_similarities(embeddings)

# Wyświetlenie wyników
print("Podobieństwo między zdjęciami:")
for img1, img2, similarity in similarities:
    print(f"{img1} vs {img2} -> Similarity: {similarity:.2f}")


Podobieństwo między zdjęciami:
ZuzanaCaputova_0001.png vs ZuzanaCaputova_0002.png -> Similarity: 0.56
ZuzanaCaputova_0001.png vs ZuzanaCaputova_0003.png -> Similarity: 0.83
ZuzanaCaputova_0001.png vs ZuzanaCaputova_0004.png -> Similarity: 0.67
ZuzanaCaputova_0001.png vs ZuzanaCaputova_0005.png -> Similarity: 0.60
ZuzanaCaputova_0001.png vs ZuzanaCaputova_0006.png -> Similarity: 0.76
ZuzanaCaputova_0001.png vs ZuzanaCaputova_0007.png -> Similarity: 0.70
ZuzanaCaputova_0001.png vs ZuzanaCaputova_0008.png -> Similarity: 0.56
ZuzanaCaputova_0002.png vs ZuzanaCaputova_0003.png -> Similarity: 0.56
ZuzanaCaputova_0002.png vs ZuzanaCaputova_0004.png -> Similarity: 0.48
ZuzanaCaputova_0002.png vs ZuzanaCaputova_0005.png -> Similarity: 0.25
ZuzanaCaputova_0002.png vs ZuzanaCaputova_0006.png -> Similarity: 0.46
ZuzanaCaputova_0002.png vs ZuzanaCaputova_0007.png -> Similarity: 0.41
ZuzanaCaputova_0002.png vs ZuzanaCaputova_0008.png -> Similarity: 0.47
ZuzanaCaputova_0003.png vs ZuzanaCaputova_0004